In [1]:
%reload_ext autoreload
%autoreload 2   

import sys
sys.path.append('../artitect/')

In [2]:
import torch
import warnings

# stop warnings
torch.set_float32_matmul_precision("high")
warnings.filterwarnings("ignore", ".*does not have many workers.*")

/opt/miniconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#autheticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient.from_config(
    credential=credential,
    path="config.json",
)

Found the config file in: config.json


In [4]:
# specify aml compute name.
cpu_compute_target = "cpu-cluster"
gpu_compute_target = "gpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("CPU compute target not found.")
    
try:
    ml_client.compute.get(gpu_compute_target)
except Exception:
    print("GPU compute target not found.")

In [7]:
from datetime import datetime
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes

name = "Train_SW_Rejection_03"

# data_asset_input = ml_client.data.get("tsdb_tser_norm", version="1")
data_asset_output = f"azureml://datastores/workspaceblobstore/paths/artifactory/rejection"
# data_asset_input = ml_client.data.get("train_dataset_tser_tsdb_normalized", version="3")
data_asset_input = ml_client.data.get("train_datasets_tser_tsdb", version="7")
# data_asset_validation = ml_client.data.get("train_datasets_tser_tsdb", version="8")
data_asset_model = ml_client.data.get("artifactory_output", version="126")

# data_asset_validation = ml_client.data.get("artifactory_validation_norm", version="3")
# data_asset_validation = ml_client.data.get("artifactory_validation_london", version="2")
# data_asset_validation = ml_client.data.get("artifactory_validation_mask_nolondon", version="2")
# data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow", version="1")
# data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow_london", version="1")
# data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow_nolondon", version="1") # only original 7 datasets
# data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow_nolondon", version="2") # all 20 datasets
data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow_nolondon", version="3") # all 20 val datasets except CiECGT

# define the command
# TODO: later - add config parameters to input
command_job = command(
    code="/workspaces/AICoE_Ramping_Artefacts/artifactory-master/src_jobs",
    command="python train_rejectionSampling_slidingWindow.py --input-path ${{inputs.tser_tsdb}} --val-path ${{inputs.val_path}} --model-path ${{inputs.model_path}} --output-path ${{outputs.output_path}}",
    environment="Pytorch112_python310_cuda10_cudnn8_ubuntu18@latest",
    inputs={
        "tser_tsdb": Input(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT,
            path=data_asset_input.id,
        ),
        "val_path": Input(
            type=AssetTypes.URI_FILE,
            mode=InputOutputModes.RO_MOUNT,
            path=data_asset_validation.id,
        ),
        "model_path": Input(
            type=AssetTypes.URI_FILE,
            mode=InputOutputModes.RO_MOUNT,
            path=data_asset_model.id,
        )
    },
    outputs={
        "output_path": Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
            path=data_asset_output, 
            name=f"artifactory_output"
        )
    },
    compute=gpu_compute_target,
    display_name=name,
    experiment_name="transformer_rejection_slidingWindow",
)

In [8]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Uploading src_jobs (0.33 MBs): 100%|██████████| 326558/326558 [00:00<00:00, 2190084.98it/s]




'https://ml.azure.com/runs/hungry_fly_r3mtcxs12f?wsid=/subscriptions/8de3e85d-b97f-48c1-a25b-5bddf9dc484c/resourcegroups/m3-mlops-dev/workspaces/m3-mlops-mlw-dev&tid=9c8f80a6-c648-4c20-90ff-5d267c8ae785'